TESIS RUINA
$\quad$
Mediante el algoritmo de Dassios y Zhao

In [1]:
import matplotlib.pyplot as plt
from scipy.stats import describe
import numpy as np
import pandas as pd
import operator
from astropy.table import QTable, Table, Column
from astropy import units as u

In [2]:
def DCP_generator2(a,rho,delta,alpha,beta,lambda0,T,N_paths,N_jumps_max):
    N_T=np.zeros(N_paths)
    for j_path in range (0,N_paths):
        T_jump=np.zeros(N_jumps_max+1)
        T_jump[0]=0
        T_self=np.zeros(N_jumps_max+1)
        T_self[0]=0
        lambda_p=np.zeros(N_jumps_max+1)
        lambda_p[0]=lambda0
        lambda_n=np.zeros(N_jumps_max+1)
        lambda_n[0]=lambda0
        for i_jump in range(0,N_jumps_max):
            E=(-1/rho)*np.log(np.random.rand())
            d=1+((delta*np.log(np.random.rand()))/(lambda_p[i_jump]-a))
            if (d>0):
                S1=(-1/delta)*np.log(d)
                S2=(-1/a)*(np.log(np.random.rand()))
                S=min(S1,S2)
            elif (d<0):
                S=(-1/a)*(np.log(np.random.rand()))
            tau=min(S,E)
            T_jump[i_jump+1]=T_jump[i_jump]+tau
            if (tau==S):
                Y_size=(-1/beta)*np.log(np.random.rand())
                T_self[i_jump+1]=1
            elif (tau==E):
                Y_size=(-1/alpha)*np.log(np.random.rand())
                T_self[i_jump+1]=0
            lambda_n[i_jump+1]=a+(lambda_p[i_jump]-a)*np.exp(-delta*tau)
            lambda_p[i_jump+1]=lambda_n[i_jump+1]+Y_size
            if T_jump[i_jump+1]>T:
                N_T[j_path]=np.sum(T_self[0:i_jump])
                break
    return N_T,T_jump,T_self,lambda_p

Se realizará la simulación para los parámetros:
$$(a,\rho,\delta,\alpha,\beta,\lambda_0,T,Njumpsmax,Npaths)=(0.7,0.5,2.0,2.0,1.5,1.0,400,10000,10000) $$

In [3]:
a=0.7;rho=0.5;delta=2.0;alpha=2.0;beta=1.5;N_jumps_max=10000;T=400;N_paths=10000;lambda0=1.0

In [ ]:
N_T,T_jump,T_self,lambda_p=DCP_generator2(a,rho,delta,alpha,beta,lambda0,T,N_paths,N_jumps_max)

Suponemos que las reclamaciones siguen distribución Exponencial de parámetro $\gamma$

In [100]:
claims=[]
i=0
for i in (N_T):
    j=0
    z=[]
    while j<i:
        y=(-1/gamma)*np.log(np.random.rand())
        z.append(y)
        j=j+1
        if j == 200000:
            break
    claim=x_0+c*T-(np.sum(z))    
    claims.append(claim)

In [102]:
claims

[-35.18726692736436]

In [101]:
i=0
count=0
while i < len(claims):
    if claims[i]<=0:
        count=count+1
    i=i+1
    if i==200000:
        break

P_ruina=(count/len(claims))        
P_ruina

1.0

Realización de la simulación de ruina con los mismo parámetros pero una prima de 0.8 y una reserva inicial de 10

In [148]:
a=0.7;delta=2.0;rho=0.5;alpha=2.0;beta=1.5;N_jumps_max=10000;T=400;N_paths=1000;gamma=1.0; c=0.8; x_0=10

In [149]:
def ruina_DCP(a,rho,delta,alpha,beta,lambda0,T,N_paths,N_jumps_max,x_0,c):
    N_T,T_jump,T_self,lambda_p=DCP_generator2(a,rho,delta,alpha,beta,lambda0,T,N_paths,N_jumps_max)
    claims=[]
    i=0
    for i in (N_T):
        j=0
        z=[]
        while j<i:
            y=(-1/gamma)*np.log(np.random.rand())
            z.append(y)
            j=j+1
            if j == 200000:
                break
        claim=x_0+c*T-(np.sum(z))    
        claims.append(claim)
    h=0
    count=0
    while h < len(claims):
        
        if claims[h]<=0:
            count=count+1
        h=h+1
        if h==200000:
            break

    P_ruina=(count/len(claims))
    return P_ruina,claims

In [142]:
P_ruina,claims=ruina_DCP(a,rho,delta,alpha,beta,lambda0,T,N_paths,N_jumps_max,x_0,c)

C:\Users\hsrc_\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  


In [143]:
P_ruina

0.053

In [150]:
P_ruina2=[]
for lambda0 in range(13):
    P_ruina,claims=ruina_DCP(a,rho,delta,alpha,beta,lambda0,T,N_paths,N_jumps_max,x_0,c)
    P_ruina2.append(P_ruina)
P_ruina2

[0.166, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]